<a href="https://colab.research.google.com/github/danielriosgarza/sharedColabBooks/blob/main/logisticModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import all the necessary packages**

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

#to make sure plotly renders in the notebook
import plotly.io as pio
pio.renderers.default = "colab"

import plotly.graph_objects as go

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
!wget https://raw.githubusercontent.com/danielriosgarza/sharedColabBooks/refs/heads/main/files/bh_od.tsv
!ls

In [ ]:
def get_experiments(fileName):
  experiments = {}

  with open(fileName) as f:
    f.readline()
    for line in f:
        a = line.strip().split('\t')
        time, experiment, replicate, OD = float(a[0]), a[1], a[2], float(a[3])
        if experiment not in experiments:
          experiments[experiment] = {}
        if time not in experiments[experiment]:
          experiments[experiment][time] = []
        experiments[experiment][time].append(OD)

  data = {}
  for experiment in experiments:
    data[experiment] = {'time': [], 'OD_mean': [], 'OD_std': []}
    for time in experiments[experiment]:
      data[experiment]['time'].append(time)
      data[experiment]['OD_mean'].append(np.mean(experiments[experiment][time]))
      data[experiment]['OD_std'].append(np.std(experiments[experiment][time]))
  return data

data = get_experiments('bh_od.tsv')

fig = go.FigureWidget()

def makeFig(data):
  for experiment in data:
      fig.add_trace(go.Scatter(
          x=data[experiment]['time'],
          y=data[experiment]['OD_mean'],
          error_y=dict(
              type='data',
              array=data[experiment]['OD_std'],
              visible=True
          ),
          mode='lines+markers',
          name=experiment))

  fig.update_layout(
      xaxis_title="Time",
      yaxis_title="OD"
  )

  fig.show()

makeFig(data)


## Logistic Growth Model

The logistic growth model is a phenomenological model that captures the essential features of population growth in a limited environment. It is particularly useful for modeling bacterial growth, where resources are finite, and the environment can only support a certain maximum population size, known as the carrying capacity.


### Mathematical Representation

For a single species $( N_i )$, the logistic growth equation is:

$\frac{dN_i}{dt} = N_i r_i \left(1 - \frac{N_i}{K_i}\right)$


Where:
- $ \frac{dN_i}{dt} $ is the rate of change of the population size over time.
- $ N_i $ is the population size of species $ i $ at time $ t $.
- $ r_i $ is the intrinsic growth rate of species $ i $.
- $ K_i $ is the carrying capacity of the environment for species $ i $.

### Extending to Multiple Species

When considering multiple species without interactions (e.g., competition or predation), each species can be modeled independently:

$\frac{dN_1}{dt} = N_1 r_1 \left(1 - \frac{N_1}{K_1}\right)$

$\frac{dN_2}{dt} = N_2 r_2 \left(1 - \frac{N_2}{K_2}\right)$

$\vdots$

$\frac{dN_i}{dt} = N_i r_i \left(1 - \frac{N_i}{K_i}\right)$

---

## Components of the Model

### States (Variables)

- **Population Sizes**: $ N_1, N_2, \ldots, N_i $
  - Represent the number of individuals in each species at any given time.
  - **Units**: Number of individuals (cells or density)

### Parameters

1. **Intrinsic Growth Rates**: $ r_1, r_2, \ldots, r_i $
   - The maximum per capita rate of increase when the population is small and resources are abundant.
   - **Units**: Inverse time (e.g., per hour, per day).
   - **Interpretation**: Determines how quickly a species can grow under ideal conditions.

2. **Carrying Capacities**: $ K_1, K_2, \ldots, K_i $
   - The maximum sustainable population size for each species in the environment.
   - **Units**: Number of individuals.
   - **Interpretation**: Reflects the limitation of resources such as nutrients, space, or light.

---



**Define the model**

In [ ]:
def logisticModel(t, N, r_values, K_values):
    dNdt = N * r_values * (1 - N / K_values)
    return dNdt


**Define the state space**

In [ ]:
# slider widget
num_species_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,#increase for more species
    step=1,
    description='SpeciesNumber',
    continuous_update=True,
    readout=True,
    readout_format='d',
)

# output widget to display the text
output = widgets.Output()

def update_output(change):
    with output:
        output.clear_output()
        print(f"Environment has {num_species_slider.value} microbial species")


num_species_slider.observe(update_output, names='value')

# Display the slider and the output
display(num_species_slider, output)

# Call the function once to display the initial output
update_output(None)


**Define the parameter values**

In [ ]:
# Output widget to hold the parameter input boxes
parameters_output = widgets.Output()

def generate_species_inputs(num_species):
    # Lists to hold the input widgets
    growth_rate_inputs = []
    carrying_capacity_inputs = []
    initial_population_inputs = []

    # Create input boxes for each species
    for i in range(num_species):
        # Growth rate input for species i
        r_input = widgets.FloatText(
            value=0.5,
            description=f'r_{i+1}:',
            step=0.01,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        growth_rate_inputs.append(r_input)

        # Carrying capacity input for species i
        K_input = widgets.IntText(
            value=100,
            description=f'K_{i+1}:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        carrying_capacity_inputs.append(K_input)

        # Initial population input for species i
        N0_input = widgets.IntText(
            value=10,
            description=f'N0_{i+1}:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        initial_population_inputs.append(N0_input)

    return growth_rate_inputs, carrying_capacity_inputs, initial_population_inputs

def update_parameters(change):
    with parameters_output:
        clear_output()
        # Get the current number of species from the slider
        num_species = num_species_slider.value
        # Generate new input boxes based on the updated number of species
        r_inputs, K_inputs, N0_inputs = generate_species_inputs(num_species)

        # Organize input boxes into boxes for better layout
        inputs_layout = []
        for i in range(num_species):
            species_label = widgets.HTML(value=f"<b>Species {i+1}</b>")
            species_inputs = widgets.VBox([species_label, r_inputs[i], K_inputs[i], N0_inputs[i]])
            inputs_layout.append(species_inputs)

        # Display input boxes in a horizontal box
        display(widgets.HBox(inputs_layout))

        # Store inputs for further use (e.g., in simulations)
        parameters_output.r_inputs = r_inputs
        parameters_output.K_inputs = K_inputs
        parameters_output.N0_inputs = N0_inputs


# Observe changes in the number of species slider
num_species_slider.observe(update_parameters, names='value')


# Initialize the parameter inputs display
update_parameters(None)

# Display the parameter inputs
display(parameters_output)

**Simulate the model**

In [ ]:
def get_parameter_values():
    # Get the current parameter values
    r_values = [input_widget.value for input_widget in parameters_output.r_inputs]
    K_values = [input_widget.value for input_widget in parameters_output.K_inputs]
    N0_values = [input_widget.value for input_widget in parameters_output.N0_inputs]
    return r_values, K_values, N0_values

def run_simulation(button, tStart = 0, tEnd = 50, tstore = 500):
    try:
        # Retrieve the parameter values
        r_values, K_values, N0_values = get_parameter_values()

        # Convert to NumPy arrays for vectorized operations
        r_values = np.array(r_values, dtype=float)
        K_values = np.array(K_values, dtype=float)
        N0_values = np.array(N0_values, dtype=float)
        num_species = num_species_slider.value

        # Check for invalid inputs
        if np.any(r_values < 0) or np.any(K_values <= 0) or np.any(N0_values < 0):
            raise ValueError("Growth rates, carrying capacities, and initial populations must be non-negative, and carrying capacities must be positive.")

        # Time span and evaluation points
        t_span = (tStart, tEnd)
        t_eval = np.linspace(t_span[0], t_span[1], tstore)

        # Solve the ODEs
        solution = solve_ivp(
            logisticModel,
            t_span,
            N0_values,
            args=(r_values, K_values),  # Pass the parameters to the model function
            t_eval=t_eval,
            method='RK45'
        )

        # Plot the results
        plt.figure(figsize=(10, 6))
        for i in range(num_species):
            plt.plot(solution.t, solution.y[i], label=f"Species {i+1}")
        plt.xlabel('Time')
        plt.ylabel('Population Size')
        plt.title('Population Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Error: {e}")

# Create a button to run the simulation
run_button = widgets.Button(
    description='Run Simulation',
    button_style='success',
    tooltip='Click to run the simulation with current parameters',
)

# Attach the simulation function to the button
run_button.on_click(run_simulation)

# Display the button
display(run_button)


**Putting it all together (to make it easier to simulate)**

we make the growth rates drawn from a random uniform distribution betweem 0 and 1, the carrying capacity a random number between 0 and 3, and concentration at $t=0$ equal to 0.01 (more of less the OD600 range).

In [ ]:

##########Model############

def logisticModel(t, N, r_values, K_values):
    dNdt = N * r_values * (1 - N / K_values)
    return dNdt

###### state-space ############

# slider widget
num_species_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=10,#increase for more species
    step=1,
    description='SpeciesNumber',
    continuous_update=True,
    readout=True,
    readout_format='d',
)

# output widget to display the text
output = widgets.Output()

def update_output(change):
    with output:
        output.clear_output()
        print(f"Environment has {num_species_slider.value} microbial species")


num_species_slider.observe(update_output, names='value')

# Display the slider and the output
display(num_species_slider, output)

# Call the function once to display the initial output
update_output(None)


#############parameter values#######

# Output widget to hold the parameter input boxes
parameters_output = widgets.Output()

def generate_species_inputs(num_species):
    # Lists to hold the input widgets
    growth_rate_inputs = []
    carrying_capacity_inputs = []
    initial_population_inputs = []

    # Create input boxes for each species
    for i in range(num_species):
        # Growth rate input for species i
        r_input = widgets.FloatText(
            value=np.random.uniform(),
            description=f'r_{i+1}:',
            step=0.01,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        growth_rate_inputs.append(r_input)

        # Carrying capacity input for species i
        K_input = widgets.FloatText(
            value=np.random.uniform(0,3),
            description=f'K_{i+1}:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        carrying_capacity_inputs.append(K_input)

        # Initial population input for species i
        N0_input = widgets.FloatText(
            value=0.001,
            description=f'N0_{i+1}:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px'),
        )
        initial_population_inputs.append(N0_input)

    return growth_rate_inputs, carrying_capacity_inputs, initial_population_inputs

def update_parameters(change):
    with parameters_output:
        clear_output()
        # Get the current number of species from the slider
        num_species = num_species_slider.value
        # Generate new input boxes based on the updated number of species
        r_inputs, K_inputs, N0_inputs = generate_species_inputs(num_species)

        # Organize input boxes into boxes for better layout
        inputs_layout = []
        for i in range(num_species):
            species_label = widgets.HTML(value=f"<b>Species {i+1}</b>")
            species_inputs = widgets.VBox([species_label, r_inputs[i], K_inputs[i], N0_inputs[i]])
            inputs_layout.append(species_inputs)

        # Display input boxes in a horizontal box
        display(widgets.HBox(inputs_layout))

        # Store inputs for further use (e.g., in simulations)
        parameters_output.r_inputs = r_inputs
        parameters_output.K_inputs = K_inputs
        parameters_output.N0_inputs = N0_inputs


# Observe changes in the number of species slider
num_species_slider.observe(update_parameters, names='value')


# Initialize the parameter inputs display
update_parameters(None)

# Display the parameter inputs
display(parameters_output)

############Simulations

def get_parameter_values():
    # Get the current parameter values
    r_values = [input_widget.value for input_widget in parameters_output.r_inputs]
    K_values = [input_widget.value for input_widget in parameters_output.K_inputs]
    N0_values = [input_widget.value for input_widget in parameters_output.N0_inputs]
    return r_values, K_values, N0_values

def run_simulation(button, tStart = 0, tEnd = 50, tstore = 500):
    try:
        # Retrieve the parameter values
        r_values, K_values, N0_values = get_parameter_values()

        # Convert to NumPy arrays for vectorized operations
        r_values = np.array(r_values, dtype=float)
        K_values = np.array(K_values, dtype=float)
        N0_values = np.array(N0_values, dtype=float)
        num_species = num_species_slider.value

        # Check for invalid inputs
        if np.any(r_values < 0) or np.any(K_values <= 0) or np.any(N0_values < 0):
            raise ValueError("Growth rates, carrying capacities, and initial populations must be non-negative, and carrying capacities must be positive.")

        # Time span and evaluation points
        t_span = (tStart, tEnd)
        t_eval = np.linspace(t_span[0], t_span[1], tstore)

        # Solve the ODEs
        solution = solve_ivp(
            logisticModel,
            t_span,
            N0_values,
            args=(r_values, K_values),  # Pass the parameters to the model function
            t_eval=t_eval,
            method='RK45'
        )

        # Plot the results
        plt.figure(figsize=(10, 6))
        for i in range(num_species):
            plt.plot(solution.t, solution.y[i], label=f"Species {i+1}")
        plt.xlabel('Time')
        plt.ylabel('Population Size')
        plt.title('Population Dynamics')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Error: {e}")

# Create a button to run the simulation
run_button = widgets.Button(
    description='Run Simulation',
    button_style='success',
    tooltip='Click to run the simulation with current parameters',
)

# Attach the simulation function to the button
run_button.on_click(run_simulation)

# Display the button
display(run_button)

